# LangChain Expression Language (LCEL)

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

## Simple chain

In [ ]:
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
output_parser = StrOutputParser()

chain = prompt | model | output_parser
chain.invoke({"topic": "bananas"})

## More complex (add retriever)

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [ ]:
vectorstore = DocArrayInMemorySearch.from_texts(
    [
        "Harrison worked at Kensho",
        "Bears like to eat honey",
    ],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

In [ ]:
retriever.invoke("Where did Harrison work?")

In [ ]:
retriever.invoke("What do bears like to eat?")

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.schema.runnable import RunnableMap

inputs = RunnableMap({
    "question": lambda x: x["question"],
    "context": lambda x: retriever.invoke(x["question"]),
})
chain = inputs | prompt | model | output_parser

print(inputs.invoke({"question": "Where did Harrison work?"}))

In [ ]:
print(chain.invoke({"question": "Where did Harrison work?"}))

## Bind (OpenAI functions)

In [ ]:
functions = [
    {
        "name": "weather_search",
        "description": "Search for weather given an airport code",
        "parameters": {
            "type": "object",
            "properties": {
                "airport_code": {
                    "type": "string",
                    "description": "The airport code to get the weather for"
                },
            },
            "required": ["airport_code"],
        },
    },
    {
        "name": "sports_search",
        "description": "Search for news of recent sport events",
        "parameters": {
            "type": "object",
            "properties": {
                "team_name": {
                    "type": "string",
                    "description": "The sports team to search for"
                },
            },
            "required": ["team_name"],
        },
    },
]

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
])
model = ChatOpenAI().bind(functions=functions)

In [ ]:
chain = prompt | model
print(chain.invoke({"input": "What's the weather is SF?"}).additional_kwargs)
print(chain.invoke({"input": "How did the Patriots do yesterday?"}).additional_kwargs)

## Fallbacks

In [ ]:
from langchain_core.language_models.fake_chat_models import FakeChatModel
import json

In [ ]:
simple_model = FakeChatModel()
simple_chain = simple_model | StrOutputParser() |json.loads

challenge = "Write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

print(simple_model.invoke(challenge))

# simple_chain.invoke(challenge) # fails because the output "fake response" can't be converted to JSON

In [ ]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

chain.invoke(challenge)

In [ ]:
final_chain = simple_chain.with_fallbacks([chain])
final_chain.invoke(challenge)

## Interface: invoke, batch, stream, async

In [ ]:
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [ ]:
chain.invoke({"topic": "gorillas"})

In [ ]:
chain.batch([
    {"topic": "fire"},
    {"topic": "horse"},
])

In [ ]:
for t in chain.stream({"topic": "glass"}):
    print(t)

In [ ]:
response = await chain.ainvoke({"topic": "bears"})
response